In [ ]:
import os
import math
import torch
from torch.autograd import Variable
from torch.optim import Adam
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader, random_split, Dataset
from scipy.io import wavfile
import scipy.signal
import numpy as np
import time
import copy

In [ ]:
NUM_CLASSES = 3
BATCH_SIZE = 1
SONG_LENGTH_SECONDS = 2

In [ ]:
class MusicDataset(Dataset):
    def __init__(self, directory, genres, add_dimension, downsample=None, noise=False):
        self.directory = directory
        self.files = []
        self.downsample = downsample
        self.add_dimension = add_dimension
        self.noise = noise
        for label, genre in enumerate(genres):
            genre_path = os.path.join(directory, genre)
            self.files.extend([(os.path.join(genre_path, f), label) for f in os.listdir(genre_path)])

    def __getitem__(self, index):
        song, label = self.files[index]
        rate, data = wavfile.read(f'{self.directory}/{song}')
        
        data = data[:44100*SONG_LENGTH_SECONDS]
        
        if self.downsample:
            data = scipy.signal.resample(data, self.downsample * SONG_LENGTH_SECONDS)

        if self.noise:
            gauss = np.random.normal(0.01, 0.001, (len(data),))
            data = data + gauss
        
        tensor = torch.Tensor(data) / (2**15)
        # add an input dimension to the data [441000] => [1, 441000]. Conv1d expects data in this format.
        if self.add_dimension:
            tensor.unsqueeze_(0)
        return tensor, torch.tensor(label, dtype=torch.long)
    
    def input_size(self):
        if self.add_dimension:
            return len(self[0][0][0])
        else:
            return len(self[0][0])
    
    def __len__(self):
        return len(self.files)



def load_dataset(add_dimension, downsample=None, noise=False):
    d = MusicDataset('.', ['rock', 'electro', 'classic'], add_dimension, downsample=downsample, noise=noise)
    train, validate = random_split(d, [900, 300])

    loader = DataLoader(train, batch_size=BATCH_SIZE)
    validation_loader = DataLoader(validate, batch_size=BATCH_SIZE)
    return d.input_size(), loader, validation_loader

In [ ]:
class Model1Linear(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.h1 = nn.Linear(input_size, hidden_size)
        self.h2 = nn.Linear(hidden_size, hidden_size)
        self.h3 = nn.Linear(hidden_size, hidden_size)
        self.h4 = nn.Linear(hidden_size, hidden_size)
        self.h5 = nn.Linear(hidden_size, hidden_size)
        self.h6 = nn.Linear(hidden_size, hidden_size)
        self.h7 = nn.Linear(hidden_size, hidden_size)
        self.h8 = nn.Linear(hidden_size, hidden_size)
        self.h9 = nn.Linear(hidden_size, NUM_CLASSES)
    
    def forward(self, x):
        x = x.data.view(-1, input_size)
         
        x = self.h1(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        x = self.h2(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        x = self.h3(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        x = self.h4(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        
        x = self.h5(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        
        x = self.h6(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        
        x = self.h7(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        x = self.h8(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        x = self.h9(x)
        x = F.softmax(x, dim=1)
        return x

In [ ]:
class ModelConv1(nn.Module):
    def __init__(self, input_size, kernel_size=5, conv_out_channels=5, linear_size=50):
        super().__init__()

        if kernel_size % 2 != 1:
            raise Exception('Only odd kernel_size are supported')
        self.conv_out_channels = conv_out_channels
        self.conv1 = nn.Conv1d(1, conv_out_channels, kernel_size=kernel_size)
        conv_layer_output_size = int(input_size - (kernel_size - 1))

        self.pooled_samples = int(conv_layer_output_size / 2)
        self.h1 = nn.Linear(self.pooled_samples * conv_out_channels, linear_size)
        self.h2 = nn.Linear(linear_size, linear_size)
        self.h3 = nn.Linear(linear_size, linear_size)
        self.h4 = nn.Linear(linear_size, linear_size)
        self.h9 = nn.Linear(linear_size, NUM_CLASSES)
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool1d(x, 2)

        x = x.view(-1, self.pooled_samples * self.conv_out_channels)

        x = self.h1(x)
        x = F.relu(x)
        
        x = self.h2(x)
        x = F.relu(x)
        
        x = self.h3(x)
        x = F.relu(x)
        
        x = self.h4(x)
        x = F.relu(x)

        x = self.h9(x)
        x = F.softmax(x, dim=1)
        return x

In [ ]:
class ModelConv2(nn.Module):
    def __init__(self, input_size, kernel_size=5, conv_out_channels=5, linear_size=50):
        super().__init__()

        if kernel_size % 2 != 1:
            raise Exception('Only odd kernel_size are supported')
        self.conv_out_channels = conv_out_channels
        self.conv1 = nn.Conv1d(1, conv_out_channels, kernel_size=kernel_size)
        self.conv2 = nn.Conv1d(conv_out_channels, conv_out_channels, kernel_size=kernel_size)
        self.conv3 = nn.Conv1d(conv_out_channels, conv_out_channels, kernel_size=kernel_size)

        #conv_layer_output_size = int(input_size - (kernel_size - 1))
        x = input_size
        x = x - (kernel_size - 1)
        x = int(x / 5)
        
        x = x - (kernel_size - 1)
        x = int(x / 5)
        
        x = x - (kernel_size - 1)
        x = int(x / 5)
        self.pooled_samples = x * conv_out_channels

        self.h1 = nn.Linear(self.pooled_samples, linear_size)
        self.h2 = nn.Linear(linear_size, linear_size)
        self.h3 = nn.Linear(linear_size, linear_size)
        self.h4 = nn.Linear(linear_size, linear_size)
        self.h9 = nn.Linear(linear_size, NUM_CLASSES)
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool1d(x, 5)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool1d(x, 5)
        
        x = self.conv3(x)
        x = F.relu(x)
        x = F.max_pool1d(x, 5)

        x = x.view(BATCH_SIZE, self.pooled_samples)
        x = self.h1(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        
        x = self.h2(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        
        x = self.h3(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        
        x = self.h4(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        x = self.h9(x)
        x = F.softmax(x, dim=1)
        return x

In [ ]:
class ModelConv2BatchNorm(nn.Module):
    def __init__(self, input_size, kernel_size=5, conv_out_channels=5, linear_size=50):
        super().__init__()

        if kernel_size % 2 != 1:
            raise Exception('Only odd kernel_size are supported')
        self.conv_out_channels = conv_out_channels
        self.conv1 = nn.Conv1d(1, conv_out_channels, kernel_size=kernel_size)
        self.batch_norm1 = nn.BatchNorm1d(conv_out_channels)
        self.conv2 = nn.Conv1d(conv_out_channels, conv_out_channels, kernel_size=kernel_size)
        self.batch_norm2 = nn.BatchNorm1d(conv_out_channels)
        self.conv3 = nn.Conv1d(conv_out_channels, conv_out_channels, kernel_size=kernel_size)
        self.batch_norm3 = nn.BatchNorm1d(conv_out_channels)

        #conv_layer_output_size = int(input_size - (kernel_size - 1))
        x = input_size
        x = x - (kernel_size - 1)
        x = int(x / 5)
        
        x = x - (kernel_size - 1)
        x = int(x / 5)
        
        x = x - (kernel_size - 1)
        x = int(x / 5)
        self.pooled_samples = x * conv_out_channels

        self.h1 = nn.Linear(self.pooled_samples, linear_size)
        self.h2 = nn.Linear(linear_size, linear_size)
        self.h3 = nn.Linear(linear_size, linear_size)
        self.h4 = nn.Linear(linear_size, linear_size)
        #self.h5 = nn.Linear(linear_size, linear_size)
        #self.h6 = nn.Linear(linear_size, linear_size)
        #self.h7 = nn.Linear(linear_size, linear_size)
        #self.h8 = nn.Linear(linear_size, linear_size)
        self.h9 = nn.Linear(linear_size, NUM_CLASSES)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.batch_norm1(x)
        x = F.relu(x)
        x = F.max_pool1d(x, 5)
        
        x = self.conv2(x)
        x = self.batch_norm2(x)
        x = F.relu(x)
        x = F.max_pool1d(x, 5)
        
        x = self.conv3(x)
        x = self.batch_norm3(x)
        x = F.relu(x)
        x = F.max_pool1d(x, 5)

        x = x.view(BATCH_SIZE, self.pooled_samples)
        x = self.h1(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        
        x = self.h2(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        
        x = self.h3(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        
        x = self.h4(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        x = self.h9(x)
        x = F.softmax(x, dim=1)
        return x

In [ ]:
class ModelConv3(nn.Module):
    def __init__(self, input_size, kernel_size=5, conv_out_channels=5, linear_size=50):
        super().__init__()

        if kernel_size % 2 != 1:
            raise Exception('Only odd kernel_size are supported')
        self.conv_out_channels = conv_out_channels
        self.conv1 = nn.Conv1d(1, conv_out_channels, kernel_size=kernel_size)
        self.conv2 = nn.Conv1d(conv_out_channels, conv_out_channels, kernel_size=kernel_size)
        self.conv3 = nn.Conv1d(conv_out_channels, conv_out_channels, kernel_size=kernel_size)

        #conv_layer_output_size = int(input_size - (kernel_size - 1))
        x = input_size
        x = x - (kernel_size - 1)
        x = int(x / 5)
        
        x = x - (kernel_size - 1)
        x = int(x / 5)
        
        x = x - (kernel_size - 1)
        x = int(x / 5)
        self.pooled_samples = x * conv_out_channels

        self.h1 = nn.Linear(self.pooled_samples, linear_size)
        self.h9 = nn.Linear(linear_size, NUM_CLASSES)
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool1d(x, 5)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool1d(x, 5)
        
        x = self.conv3(x)
        x = F.relu(x)
        x = F.max_pool1d(x, 5)

        x = x.view(BATCH_SIZE, self.pooled_samples)
        x = self.h1(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        x = self.h9(x)
        x = F.softmax(x, dim=1)
        return x

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, device, num_epochs=25):
    since = time.time()

    val_acc_history = []

    #best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    f = open(f'{datetime.now().isoformat()}.txt', 'w', buffering=1)


    for epoch in range(num_epochs):
        train_loss = None
        train_accuracy = None

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            if phase == 'train':
                train_loss = epoch_loss
                train_accuracy = epoch_acc
            else:
                stats = f'Epoch: {epoch}, TL: {train_loss:.5f}, VL: {epoch_loss:.5f}, TA: {train_accuracy:.5f}, VA: {epoch_acc:.5f}'
                print(stats)
                f.write(f'{stats}\n')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                #best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), f'{best_acc:.5f}.pth')
                
            if phase == 'val':
                val_acc_history.append(epoch_acc)


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

input_size, loader, validation_loader = load_dataset(add_dimension=True, downsample=22050, noise=True)
model = ModelConv2(input_size, kernel_size=5, conv_out_channels=10, linear_size=500).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = Adam(params=model.parameters(), lr=0.0001)
train_model(model, {'train': loader, 'val': validation_loader}, criterion, optimizer, device, num_epochs=10000)

In [ ]:
#input_size, loader, validation_loader = load_dataset(add_dimension=False, noise=False)
#model = Model1Linear(input_size, 500).cuda()
#learn(model, loader, validation_loader, 10000, learning_rate=0.0001)

In [ ]:
#input_size, loader, validation_loader = load_dataset(add_dimension=True, noise=True)
#model = ModelConv3(input_size, kernel_size=5, conv_out_channels=10, linear_size=500).cuda()
#learn(model, loader, validation_loader, epochs=10000, learning_rate=0.0001)